# Tutorial 2: Custom Models and Training

This tutorial demonstrates how to define a custom Graph Convolutional Network (GCN)
and integrate it into a ZenML pipeline.


In [1]:
import torch
import pytorch_lightning as pl
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from zenml import pipeline, step

from pioneerml.common.models.base import GraphModel
from pioneerml.common.pipeline.services.training.utils import GraphLightningModule
from pioneerml.pipelines.tutorial_examples.graph_datamodule import GraphDataModule
from pioneerml.common.zenml.materializers import (
    PyGDataListMaterializer,
)
from pioneerml.common.zenml import load_step_output
from pioneerml.common.zenml import utils as zenml_utils
from pioneerml.common.zenml.utils import detect_available_accelerator

# Initialize ZenML for notebook use
# setup_zenml_for_notebook automatically finds the project root by searching
# upward for .zen or .zenml directories, ensuring we use the root configuration.
# use_in_memory=True creates a temporary in-memory SQLite store, perfect for
# tutorials where we don't need persistent artifact storage.
zenml_client = zenml_utils.setup_zenml_for_notebook(use_in_memory=True)
print(f"ZenML initialized with stack: {zenml_client.active_stack_model.name}")


Using ZenML repository root: /workspace
Ensure this is the top-level of your repo (.zen must live here).
ZenML initialized with stack: default


## Define a Custom Graph Convolutional Network

We'll create a simple but effective GCN model for graph classification. The architecture
consists of:

1. **Two GCN layers**: Each layer aggregates information from neighboring nodes
   - First layer: Projects 5D node features → hidden dimension (64)
   - Second layer: Further refines node representations within the hidden space

2. **Global mean pooling**: Aggregates all node features into a single graph-level
   representation by taking the mean across all nodes

3. **Classifier head**: A linear layer that maps the graph representation to class
   logits (3 classes: π, μ, e+)

**Why this architecture?** GCNs are excellent for learning from graph-structured data
because they respect the graph topology - nodes learn representations based on their
neighbors. Global pooling allows us to make graph-level predictions from node features.


In [2]:
class SimpleGCN(GraphModel):
    """A simple custom Graph Convolutional Network for graph classification.
    
    This model demonstrates the core components of a GCN:
    - Graph convolution layers that aggregate neighbor information
    - Global pooling to create graph-level representations
    - A classifier head for final predictions
    """
    
    def __init__(self, num_classes: int = 3, hidden_dim: int = 64):
        super().__init__()
        
        # First GCN layer: projects 5D node features to hidden dimension
        # GCNConv learns to aggregate information from each node's neighbors
        self.conv1 = GCNConv(5, hidden_dim)
        
        # Second GCN layer: refines node representations within hidden space
        # This allows the model to learn more complex patterns in the graph structure
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        
        # Classifier head: maps graph-level representation to class logits
        # Output size matches number of classes (3 for π, μ, e+)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, batch):
        """Forward pass through the GCN.
        
        Args:
            batch: PyTorch Geometric Batch object containing:
                - x: Node features [num_nodes, 5]
                - edge_index: Edge connectivity [2, num_edges]
                - batch: Batch assignment vector [num_nodes]
        
        Returns:
            Logits for each graph in the batch [batch_size, num_classes]
        """
        # Extract graph components from the batch
        x, edge_index, batch_indices = batch.x, batch.edge_index, batch.batch

        # First graph convolution: aggregate neighbor features
        # ReLU activation introduces non-linearity
        x = self.conv1(x, edge_index).relu()
        
        # Second graph convolution: further refine node representations
        x = self.conv2(x, edge_index).relu()

        # Global mean pooling: aggregate all node features into a single vector
        # This creates one representation per graph in the batch
        # Shape: [batch_size, hidden_dim]
        x = global_mean_pool(x, batch_indices)

        # Final classification: map graph representation to class logits
        # Shape: [batch_size, num_classes]
        return self.classifier(x)


    def export_torchscript(self, path=None, *, strict: bool = False):
        """Export a TorchScript module for this model."""
        scripted = torch.jit.script(self.eval().cpu())
        if path is not None:
            scripted.save(str(path))
        return scripted


# Verify the model can be instantiated
model = SimpleGCN(num_classes=3, hidden_dim=64)
print(f"Custom GCN model created: {model.num_parameters:,} parameters")


Custom GCN model created: 4,739 parameters


## Build the Training Pipeline

Now we'll create a complete ZenML pipeline that uses our custom GCN model. The pipeline
follows the same structure as Tutorial 1:

1. **`create_data`**: Generates synthetic graph data with class labels
2. **`create_datamodule`**: Splits data into train/validation sets
3. **`create_model`**: Instantiates our custom SimpleGCN
4. **`create_lightning_module`**: Wraps the model in PyTorch Lightning
5. **`train_model`**: Executes the training loop

**Key difference from Tutorial 1**: Here we use our custom `SimpleGCN` instead of the
pre-built `GroupClassifier`. Everything else works exactly the same - this demonstrates
the flexibility of the ZenML + GraphModel architecture.


In [3]:
def create_simple_synthetic_data(num_samples: int = 150) -> list[Data]:
    """
    Generate synthetic graph data with learnable structure.
    
    Each class has:
      - different mean patterns in feature space
      - slightly different edge density
      - noise added so the classifier must generalize
    """
    # Define distinct class centers for the 5D node features
    class_centers = torch.tensor([
        [ 1.0, -1.0,  0.5,  0.0,  2.0],   # Class 0 (π)
        [-1.0,  0.5,  1.5,  1.0, -1.0],   # Class 1 (μ)
        [ 0.0,  1.0, -1.0, -0.5,  1.5],   # Class 2 (e+)
    ])

    data = []

    for _ in range(num_samples):
        num_nodes = torch.randint(4, 9, (1,)).item()

        # Choose class label
        label = torch.randint(0, 3, (1,)).item()
        y = torch.zeros(3)
        y[label] = 1.0

        # Create node features centered around class-specific mean
        # Add moderate noise so the model must learn to separate distributions
        center = class_centers[label]
        x = center + 0.3 * torch.randn(num_nodes, 5)

        # Class-dependent graph density (optional structure signal)
        edge_factor = {0: 1.0, 1: 1.3, 2: 0.7}[label]
        num_edges = int(num_nodes * edge_factor * 2)

        edge_index = torch.randint(0, num_nodes, (2, num_edges))
        edge_attr = torch.randn(num_edges, 4)

        data.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y))

    return data



@step(output_materializers=PyGDataListMaterializer, enable_cache=False)
def create_data() -> list[Data]:
    """Step 1: Generate synthetic graph data.
    
    The PyGDataListMaterializer ensures efficient serialization of PyTorch Geometric
    Data objects, avoiding pickle warnings.
    """
    return create_simple_synthetic_data()


@step(enable_cache=False)
def create_datamodule(data: list[Data]) -> GraphDataModule:
    """Step 2: Create data module with train/val split.
    
    - val_split=0.3: 70% train, 30% validation
    - batch_size=16: smaller batches for the custom model
    """
    return GraphDataModule(dataset=data, val_split=0.3, batch_size=16, num_workers=0)


@step
def create_model(num_classes: int = 3, hidden_dim: int = 64) -> SimpleGCN:
    """Step 3: Instantiate our custom GCN model.
    
    This step creates the SimpleGCN we defined above with the specified
    number of classes and hidden dimension.
    """
    return SimpleGCN(num_classes=num_classes, hidden_dim=hidden_dim)


@step
def create_lightning_module(model: SimpleGCN) -> GraphLightningModule:
    """Step 4: Wrap model in PyTorch Lightning module.
    
    Adds training logic, loss function (BCE for multilabel classification),
    and optimizer configuration.
    """
    return GraphLightningModule(model, task="classification", lr=5e-4)


@step
def train_model(
    lightning_module: GraphLightningModule,
    datamodule: GraphDataModule
) -> GraphLightningModule:
    """Step 5: Execute training loop.
    
    Automatically detects available hardware (CPU/GPU) and runs for 5 epochs.
    Returns the trained module in eval mode.
    """
    import pytorch_lightning as pl

    accelerator, devices = detect_available_accelerator()

    trainer = pl.Trainer(
        accelerator=accelerator,
        devices=devices,
        max_epochs=5,
        logger=False,
        enable_checkpointing=False,
        enable_progress_bar=False,
    )

    trainer.fit(lightning_module, datamodule=datamodule)
    return lightning_module.eval()


@pipeline
def custom_model_pipeline():
    """Compose all steps into a complete training pipeline.
    
    ZenML automatically wires the outputs of one step to the inputs of the next
    based on parameter names. For example, `create_datamodule(data)` receives
    the output from `create_data()`.
    """
    data = create_data()
    datamodule = create_datamodule(data)
    model = create_model()
    lightning_module = create_lightning_module(model)
    trained_module = train_model(lightning_module, datamodule)
    return trained_module, datamodule


## Run the Pipeline

Execute the pipeline and load the trained model. After the pipeline completes, we load
the trained model and datamodule using `load_step_output`. These artifacts are stored
by ZenML and can be reloaded anytime without re-running the pipeline.

**Why use `enable_cache=False`?** This ensures the pipeline runs fresh each time,
which is useful for tutorials. In production, you'd typically enable caching to
skip re-running unchanged steps.


In [4]:
run = custom_model_pipeline.with_options(enable_cache=False)()
print(f"Pipeline run status: {run.status}")

# Load artifacts from the run
trained_module = load_step_output(run, "train_model")
datamodule = load_step_output(run, "create_datamodule")
custom_model = load_step_output(run, "create_model")

if trained_module is None or datamodule is None or custom_model is None:
    raise RuntimeError("Could not load artifacts from the custom_model_pipeline run.")

# Move model to best available device
# Note: Models saved by ZenML may be on CPU. We move to GPU if available
# to match the training device and speed up inference.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_module = trained_module.to(device).eval()
datamodule.setup(stage="fit")
print(f"Loaded artifacts from run {run.name} (device={device})")


Initiating a new run for the pipeline: custom_model_pipeline.
Caching is disabled by default for custom_model_pipeline.
Using user: default
Using stack: default
  deployer: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step create_data has started.
Step create_data has finished in 0.471s.
Step create_model has started.
[create_model] No materializer is registered for type <class '__main__.SimpleGCN'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class '__main__.SimpleGCN'> according to the instructions at https://docs.zenml.io/concepts/artifacts/materializers
Step create_model has finished in 0.080s.
Step create_datamodule has started.
[create_dat

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
You are using a CUDA device ('NVIDIA GeForce RTX 5070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model   │ SimpleGCN         │  4.7 K │ train │     0 │
│ 1 │ loss_fn │ BCEWithLogitsLoss │      0 │ train │     0 │
└───┴─────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 4.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.7 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 9                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

[train_model] /opt/conda/envs/pioneerml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the num_workers argument to num_workers=11 in the DataLoader` to improve performance.

[train_model] /opt/conda/envs/pioneerml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the num_workers argument to num_workers=11 in the DataLoader` to improve performance.



`Trainer.fit` stopped: `max_epochs=5` reached.


[train_model] No materializer is registered for type <class 'pioneerml.common.pipeline.services.training.utils.graph_lightning_module.GraphLightningModule'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'pioneerml.common.pipeline.services.training.utils.graph_lightning_module.GraphLightningModule'> according to the instructions at https://docs.zenml.io/concepts/artifacts/materializers
Step train_model has finished in 2.320s.
Pipeline run has finished in 5.167s.
Pipeline run status: completed
Loaded artifacts from run custom_model_pipeline-2026_02_19-19_11_24_478206 (device=cuda)


## Inspect the Custom Model

Let's examine the trained model to understand its structure and verify it trained
correctly. We'll check:

- **Parameter count**: How many trainable parameters does our custom GCN have?
- **Input/output shapes**: What dimensions does the model expect and produce?
- **Device placement**: Where is the model running (CPU/GPU)?

This helps validate that the model architecture is correct and that training completed
successfully.


In [5]:
device = next(trained_module.parameters()).device
param_count = sum(p.numel() for p in trained_module.parameters())

# Get a sample batch to inspect input shapes
train_loader = datamodule.train_dataloader()
first_batch = next(iter(train_loader))

print("Custom GCN Model Summary:")
print(f"- Run: {run.name}")
print(f"- Device: {device}")
print(f"- Total parameters: {param_count:,}")
print(f"- Input node features: {first_batch.x.shape[1]}D (shape: {tuple(first_batch.x.shape)})")
print(f"- Number of nodes in batch: {first_batch.x.shape[0]}")
print(f"- Number of edges: {first_batch.edge_index.shape[1]}")

# Test forward pass to verify output shape
with torch.no_grad():
    output = trained_module(first_batch.to(device))
    print(f"- Output logits shape: {tuple(output.shape)} (batch_size, num_classes)")


Custom GCN Model Summary:
- Run: custom_model_pipeline-2026_02_19-19_11_24_478206
- Device: cuda:0
- Total parameters: 4,739
- Input node features: 5D (shape: (104, 5))
- Number of nodes in batch: 104
- Number of edges: 191
- Output logits shape: (16, 3) (batch_size, num_classes)


## Evaluate Model Performance

Finally, let's compute validation accuracy to confirm the custom model trained
successfully. We'll run inference on the validation set and compare predictions
to ground truth labels.

**What we're measuring**: Classification accuracy - the percentage of graphs
correctly classified into their true class (π, μ, or e+).

**Note on device placement**: The model was trained on GPU (as shown in the pipeline
output), but when loaded from ZenML artifacts, it may be on CPU. We explicitly move
it to the best available device (GPU if available) to match training conditions and
speed up inference. This is why you see the device change from CPU to GPU (or vice versa)
depending on your hardware.


In [6]:
# Get validation loader (fallback to train if val is empty)
val_loader = datamodule.val_dataloader()
if isinstance(val_loader, list) and len(val_loader) == 0:
    val_loader = datamodule.train_dataloader()

# Compute accuracy
correct = 0
total = 0
trained_module.eval()

for batch in val_loader:
    batch = batch.to(device)
    with torch.no_grad():
        logits = trained_module(batch)
    
    # Handle labels: PyTorch Geometric batches graph-level labels
    # Each graph has y of shape (num_classes,), batched into (batch_size, num_classes)
    labels = batch.y
    
    # Ensure labels are 2D: (batch_size, num_classes)
    if labels.dim() == 1:
        # If flattened, reshape assuming num_classes=3
        if labels.shape[0] % 3 == 0:
            labels = labels.view(-1, 3)
        else:
            # Single graph case: add batch dimension
            labels = labels.unsqueeze(0)
    
    # Convert one-hot encoded labels to class indices
    # Shape: (batch_size, num_classes) -> (batch_size,)
    labels = torch.argmax(labels, dim=1)
    
    # Get predicted class (highest logit)
    # Shape: (batch_size, num_classes) -> (batch_size,)
    preds = torch.argmax(logits, dim=1)
    
    # Count correct predictions (both should be shape (batch_size,))
    correct += int((preds == labels).sum().item())
    total += int(labels.numel())

accuracy = correct / total if total > 0 else 0.0
print(f"Validation accuracy: {accuracy:.1%} ({correct}/{total} correct)")


Validation accuracy: 93.3% (42/45 correct)
